<a href="https://colab.research.google.com/github/tproffen/ORCSGirlsPython/blob/master/Numbers/LogisticalMapChaosSolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*italicized text*<img src="https://github.com/tproffen/ORCSGirlsPython/blob/master/Images/Logo.png?raw=1" width="10%" align="right" hpsace="50">

# Fun with Equations

Let us play with  numbers and unlock the secrets of the logistical map. Please make sure you run the cell with the setup commands below before running any other code in this notebook.


In [ ]:
!curl -s -o setup.sh https://raw.githubusercontent.com/tproffen/ORCSGirlsPython/master/Numbers/Helpers/setup_activities.sh
!bash setup.sh
from Helpers.helpers import *

## Simple line

Before we start with the actual equation for the logistical map, let us look at a simple line. The equation for a line is simply

$y=m*x + b$

where $m$ is the slope and $b$ is the y-axis intercept.

In [ ]:
def calculateLine(m,b):
  xmin = -5.0                        # Smallest x value to calculate the function
  xmax =  5.0                        # Largest x value to calculate the function
  dx = 0.05                          # Step size in x

  xvals = []                         # Empty list for x values
  yvals = []                         # Empty list for y values

  for x in np.arange(xmin,xmax,dx):  # Loop over points in x

      y = m*x + b                    # Calculate the y-value

      xvals.append(x)                # Add x valaue to the list
      yvals.append(y)                # Add y valaue to the list

  return xvals,yvals                 # Return the values

Now we can use the function to make our plot. In order to explore our functions, we can add sliders to change the values of $m$ and $b$ and see how th egraph changes. Below is the code. Feel free to make changes to the vaklues or limits.

In [ ]:
sliders = [
  PlotSlider(name="m",value=1,max_value=10,min_value=-10,step_value=0.1),
  PlotSlider(name="b",value=1,max_value=10,min_value=-10,step_value=0.1)
]
graph = InteractivePlot(calculateLine,sliders)
graph.show()

## Now the fun - the Logistic Map

We have discussed the logistical map in the class. There is also <a href="https://youtu.be/ovJcsL7vyrk">this video</a> we saw parts of in class.

Here is the equation:
$x_{n+1} = r x_{n}(1 - x_{n})$

Following the notation used in class, $r$ stands for the growth rate and $x$ is population. This is not the actual number, but the fraction of the maximum possible population, so it ranges from 0 to 1.

#### Step 1: Visualize the function

Let's define a function `calculateGrowth` and plot how the population next year depends on the population this year. It is very similar to the line, just the line with the equation is different and we use a different range in $x$ to visualize.

In [ ]:
def calculateGrowth(r):
  xmin = 0.0                         # Smallest x value to calculate the function
  xmax = 1.0                         # Largest x value to calculate the function
  dx = 0.01                          # Step size in x

  xvals = []                         # Empty list for x values
  yvals = []                         # Empty list for y values

  for x in np.arange(xmin,xmax,dx):  # Loop over points in x

      y = r*x*(1-x)                  # Calculate the y-value

      xvals.append(x)                # Add x valaue to the list
      yvals.append(y)                # Add y valaue to the list

  return xvals,yvals                 # Return the values

Like before, we can create an interactive plot with sliders. In this case we are just varying the growth rate $r$.

In [ ]:
sliders = [
  PlotSlider(name="r",value=2.0,max_value=3.0,min_value=0.0,step_value=0.05),
]
graph = InteractivePlot(calculateGrowth,sliders)
graph.xtitle = "x_n"
graph.ytitle = "x_n+1"
graph.ymax = 0.7
graph.show()

Woohoo, just like in the video. We have a 'single hump function' with a negative feedback, meaning when the population gets too big the dowturn of the parabula will reduce the population as you can see on the graph.

#### Step 2: Behaviour as time goes on

We are most interested in the long term behaviour of the function as we repeatley call it. In other words what happens to the population after a long time. In the function below we calculate the value of $x_{n} = r x_{n-1}(1 - x_{n-1})$ for all $n$. Note that we have rewritten this so the value we calculate os $n$ and it depends on the earlier value $n-1$ which is how we implement it by using the list element `[-1]`. We also pass an argument $nmax$ which determined for how many generations we canculate the function. Now rather that calculating the value in one line, we *iterate* over the logistical map function $nmax$ times and record the result as `yvals` for each iteration.

Below is the function `calculateGrowthTime`.

In [ ]:
def calculateGrowthTime(x0, r,nmax):
  nmax = int(nmax)

  # First point
  yvals=[x0]
  xvals=[0]

  for i in range(1,nmax):                    # Loop to the maximum number nmax we want.
    xvals.append(i)
    yvals.append(r*yvals[-1]*(1-yvals[-1]))  # Append the calculated value

  return xvals,yvals                         # Return all values

And as before we now create the interactive plot. Note we now have three parameters we want to vary: the initial popuation $x0$, the growth rate $r$ and the number of generation $nmax$ which you can think of as time it like in the example we think of it as population per year.

In [ ]:
sliders = [
  PlotSlider(name="x0",  value=0.4,max_value=0.9,min_value=0.1,step_value=0.05),
  PlotSlider(name="r",   value=2.2,max_value=4.0,min_value=0.5,step_value=0.05),
  PlotSlider(name="nmax",value=200,max_value=500,min_value= 20,step_value=0.05),
]
graph = InteractivePlot(calculateGrowthTime,sliders)
graph.xtitle = "Time"
graph.ytitle = "Population"
graph.ymax = 1.0
graph.ymin = 0.0
graph.mode = "lines+markers"

graph.show()


**Stop here and explore**

<hr>
Explore how the function depends on `x0` and `r`. Anything spooky 😨 You might need to turn on autoscale (one of the buttons on top of the plot) is the function disappears from view. Share findings in the chat.
<hr>


#### Step 3: Exploring as function of growth rate $r$

You might have seen strange behaviour for certain values of $r$, so next we explore the resulting populations after some time. Basically we want to see the limit values for the population as a function or growth rate $r$.

Because there are multiple values the function oscillates around, we simply loop over $r$ and then plot the last `nplot` points in a scatter plot.

Below is the code for the calculation. It calls the function `calculateGrowthTime` you completed earlier. But now the loop is over $r$ and we keep the points from index `nplot` which should be large enough so the function has settled in its behavior. For the plotting to work we need to resort the lists which is done with the `np.array(vals).ravel()` commands.

In [ ]:
x0 = 0.4                  # Starting population
nmax  = 300               # Maximum value of n for calculate
nplot = 200               # First point to plot, so we plot nplot:nmax
rmin  = 2.9               # Starting growth rate r for graph
rmax  = 4.0               # Ending growth rate r for graph
npts  = 2000              # Number of points

dr = (rmax-rmin) / npts

rvals=[]         # List for values of r
lvals=[]         # List for resulting values of x (there will be nmax values for each r)

for r in np.arange(rmin,rmax,dr):                             # Loop over r
    lvals.append(calculateGrowthTime(x0,r,nmax)[1][nplot:]) # Getting list of x starting at nplot
    rvals.append([r]*(nmax-nplot))                            # We need the same number of r's so
                                                              # lists are the same length

# Now we need to turn the list of lists into one contineous list for plotting
lvals = np.array(lvals).ravel()
rvals = np.array(rvals).ravel()

This might take a moment. Let us see how many data points we have created.

In [ ]:
print (len(rvals))

All that is left is to plot. Adjust marker size and color below. Also explore changing nmax, nplot and the step size in r above and rerun the cells. Enjoy.

In [ ]:
layout = go.Layout(xaxis_title='Growth rate r', yaxis_title='Population limit', width=800, height=600)
data = go.Scattergl(x=rvals, y=lvals, mode='markers', marker=dict(size=1, color='red'))
graph = go.FigureWidget(data=[data], layout=layout)

In [ ]:
display(graph)


You can save the image and show it to everyone. Not only have you explored the logistical map (that is what the equation is called), but you also created **chaos**.